# Üniversite Kampüsü Ring Otobüsü Rota Optimizasyonu - Karınca Kolonisi Algoritması

## Öğrenci Bilgileri

**Adınız:** [Buraya adınızı yazın]

**Soyadınız:** [Buraya soyadınızı yazın]

**Okul Numaranız:** [Buraya okul numaranızı yazın]

**GitHub Repo Bağlantısı:** [Buraya GitHub repo linkinizi yazın]

---

## Proje Hakkında

Bu proje, bir üniversitenin kampüsü içinde ring seferi yapan otobüsün **Karınca Kolonisi Algoritması (ACO)** kullanılarak en kısa ve verimli tur rotasını bulmayı hedefler. Otobüs, kampüs içindeki fakülteler, yurtlar ve spor kompleksi gibi **10 farklı duraktan** geçerek başlangıç noktasına geri döner.

### Hedef Duraklar:
1. Mühendislik Fakültesi
2. İktisadi ve İdari Bilimler Fakültesi
3. Fen-Edebiyat Fakültesi
4. Tıp Fakültesi
5. Merkezi Kütüphane
6. Öğrenci Yurdu A
7. Öğrenci Yurdu B
8. Spor Kompleksi
9. Teknokent
10. Rektörlük Binası



In [ ]:
# Gerekli kütüphaneleri yükleme
!pip install googlemaps -q

import numpy as np
import matplotlib.pyplot as plt
import math
import googlemaps
from typing import List, Tuple, Optional
from dataclasses import dataclass
from getpass import getpass
import os

print("✅ Tüm kütüphaneler başarıyla yüklendi!")



## 1. Google Maps API Key Ayarlama (Güvenli Yöntem)

**Güvenlik Notu:** API key'iniz GitHub'a yüklenmeyecek!

### Yöntem 1: Colab Secrets (Önerilen - En Güvenli)
1. Sol menüden 🔑 (key) ikonuna tıklayın
2. "Add secret" butonuna tıklayın
3. Name: `GOOGLE_MAPS_API_KEY`
4. Value: API key'inizi yazın
5. "Add" butonuna tıklayın

### Yöntem 2: Manuel Giriş (Alternatif)
Çalışma zamanında gizli olarak gireceksiniz

**Not:** Google Maps API kullanmak için Google Cloud Console'da billing aktif olmalıdır.



In [ ]:
# Google Maps API Key - Güvenli Yöntem
# ⚠️ API key GitHub'a yüklenmeyecek!

# Yöntem 1: Colab Secrets'tan oku (en güvenli - önerilen)
try:
    from google.colab import userdata
    GOOGLE_MAPS_API_KEY = userdata.get('GOOGLE_MAPS_API_KEY', '')
    if GOOGLE_MAPS_API_KEY:
        print("✅ API key Colab Secrets'tan yüklendi (güvenli)")
    else:
        raise Exception("Secret bulunamadı")
except:
    # Yöntem 2: Environment variable'dan oku
    GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY', '')
    if GOOGLE_MAPS_API_KEY:
        print("✅ API key environment variable'dan yüklendi")
    else:
        # Yöntem 3: Kullanıcıdan gizli input al
        print("🔐 API key'inizi girin (gizli olarak):")
        print("   (Colab Secrets kullanmak için: Sol menüden 🔑 ikonuna tıklayın)")
        GOOGLE_MAPS_API_KEY = getpass("Google Maps API Key: ")

# API key kontrolü
if not GOOGLE_MAPS_API_KEY or not GOOGLE_MAPS_API_KEY.strip():
    raise RuntimeError("Google Maps API key bulunamadı!")

print(f"✅ API key bulundu: {GOOGLE_MAPS_API_KEY[:10]}...")



## 2. ACO Konfigürasyon Sınıfı

Karınca Kolonisi Algoritması için parametre yönetimi.



In [ ]:
@dataclass
class ACOConfig:
    """Ant Colony Optimization algoritması için yapılandırma sınıfı."""
    n_ants: int = 20
    n_iterations: int = 100
    alpha: float = 1.0  # Feromon etkisi
    beta: float = 5.0  # Mesafe etkisi
    evaporation_rate: float = 0.5
    q: float = 100.0  # Feromon güncelleme sabiti
    random_seed: Optional[int] = None

print("✅ ACOConfig sınıfı tanımlandı!")



## 3. Okul Koordinatları

Bursa'daki 12 lisenin koordinatları.



In [ ]:
# Kampüs içindeki ring otobüsü duraklarının koordinatları (temsili)
school_coordinates = {
    "Mühendislik Fakültesi": {"lat": 40.2255, "lng": 28.8821},
    "İktisadi ve İdari Bilimler Fakültesi": {"lat": 40.2238, "lng": 28.8854},
    "Fen-Edebiyat Fakültesi": {"lat": 40.2221, "lng": 28.8802},
    "Tıp Fakültesi": {"lat": 40.2203, "lng": 28.8879},
    "Merkezi Kütüphane": {"lat": 40.2242, "lng": 28.8785},
    "Öğrenci Yurdu A": {"lat": 40.2210, "lng": 28.8768},
    "Öğrenci Yurdu B": {"lat": 40.2194, "lng": 28.8829},
    "Spor Kompleksi": {"lat": 40.2178, "lng": 28.8796},
    "Teknokent": {"lat": 40.2270, "lng": 28.8872},
    "Rektörlük Binası": {"lat": 40.2230, "lng": 28.8830},
}

school_names = list(school_coordinates.keys())
print(f"✅ {len(school_coordinates)} durak koordinatı yüklendi!")
for i, (name, coords) in enumerate(school_coordinates.items(), 1):
    print(f"{i:2d}. {name}: ({coords['lat']:.4f}, {coords['lng']:.4f})")



## 4. Google Maps API ile Mesafe Matrisi Oluşturma

Gerçek sürüş mesafelerini Google Maps Distance Matrix API'den alıyoruz.



In [ ]:
# Google Maps API ile gerçek sürüş mesafelerini al
# ⚠️ MAX_ELEMENTS_EXCEEDED hatasını önlemek için parçalara bölerek çağrı yapıyoruz
print("🔄 Google Maps API'den mesafe matrisi çekiliyor...")
print("   (Bu işlem birkaç saniye sürebilir - 12x12 = 144 mesafe hesaplanıyor)")
print("   ⚠️ API limiti nedeniyle parçalara bölerek çağrı yapılıyor (maksimum 100 element/request)")

try:
    client = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
    
    locations = [(coords["lat"], coords["lng"]) for coords in school_coordinates.values()]
    
    n = len(school_names)
    distance_matrix = np.zeros((n, n), dtype=float)
    
    # Google Maps API limiti: maksimum 100 element per request
    # 12x12 = 144 element olduğu için parçalara bölmemiz gerekiyor
    # Her 8 origin için bir çağrı yapalım (8x12 = 96 element < 100)
    batch_size = 8
    num_batches = (n + batch_size - 1) // batch_size
    
    print(f"   📊 {num_batches} parça halinde API çağrısı yapılacak...")
    
    # Matrisi parçalara bölerek API çağrıları yap
    for batch_idx, start_idx in enumerate(range(0, n, batch_size), 1):
        end_idx = min(start_idx + batch_size, n)
        origins_batch = locations[start_idx:end_idx]
        
        print(f"   🔄 Parça {batch_idx}/{num_batches}: {start_idx+1}-{end_idx}. satırlar işleniyor...")
        
        # Her batch için tüm destination'ları kullan
        response = client.distance_matrix(
            origins=origins_batch,
            destinations=locations,  # Tüm destination'lar
            mode="driving",
            units="metric",
            region="tr",
        )
        
        rows = response.get("rows", [])
        if len(rows) != (end_idx - start_idx):
            raise RuntimeError(f"Distance Matrix API beklenmeyen bir cevap döndürdü. Beklenen {end_idx - start_idx} satır, alınan {len(rows)} satır.")
        
        # Matrisi doldur
        for batch_row_idx, row in enumerate(rows):
            i = start_idx + batch_row_idx
            elements = row.get("elements", [])
            if len(elements) != n:
                raise RuntimeError(
                    f"Distance Matrix API satır sayısı ile sütun sayısı uyumsuz. "
                    f"Satır {i}: beklenen {n} element, alınan {len(elements)} element."
                )
            
            for j, element in enumerate(elements):
                status = element.get("status")
                if status == "OK":
                    distance_meters = element["distance"]["value"]
                    distance_matrix[i, j] = distance_meters / 1000.0  # metre -> kilometre
                else:
                    distance_matrix[i, j] = 1e9  # Erişilemeyen konumlar
        
        print(f"   ✅ Parça {batch_idx}/{num_batches} tamamlandı ({end_idx - start_idx}x{n} = {(end_idx - start_idx)*n} element)")
    
    np.fill_diagonal(distance_matrix, 0.0)
    
    print(f"\n✅ Mesafe matrisi başarıyla oluşturuldu: {distance_matrix.shape}")
    print(f"   Minimum mesafe: {np.min(distance_matrix[distance_matrix > 0]):.2f} km")
    print(f"   Maksimum mesafe: {np.max(distance_matrix):.2f} km")
    print(f"   Ortalama mesafe: {np.mean(distance_matrix[distance_matrix > 0]):.2f} km")
    
except googlemaps.exceptions.ApiError as e:
    error_msg = str(e)
    if "REQUEST_DENIED" in error_msg or "billing" in error_msg.lower():
        raise RuntimeError(
            f"Google Maps API hatası: {error_msg}\n\n"
            f"Lütfen Google Cloud Console'da billing'i aktifleştirin: "
            f"https://console.cloud.google.com/project/_/billing/enable"
        )
    else:
        raise RuntimeError(f"Google Maps API hatası: {error_msg}")
except Exception as e:
    raise RuntimeError(f"Mesafe matrisi oluşturulamadı: {str(e)}")



In [ ]:
# Mesafe matrisini görselleştir
plt.figure(figsize=(10, 8))
plt.imshow(distance_matrix, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Mesafe (km)')
plt.title('Okullar Arası Mesafe Matrisi (Google Maps API)', fontsize=14, fontweight='bold')
plt.xlabel('Okul İndeksi')
plt.ylabel('Okul İndeksi')
plt.xticks(range(len(school_names)), range(len(school_names)))
plt.yticks(range(len(school_names)), range(len(school_names)))
plt.tight_layout()
plt.show()



## 5. Karınca Kolonisi Optimizasyon Algoritması

Tam ACO algoritması implementasyonu.



In [ ]:
class AntColonyOptimizer:
    """Karınca Kolonisi Algoritması (ACO) ile TSP tabanlı rota optimizasyonu."""
    
    def __init__(self, config: ACOConfig):
        self.config = config
        if config.random_seed is not None:
            np.random.seed(config.random_seed)
    
    def _initialize_pheromones(self, n_cities: int) -> np.ndarray:
        """Başlangıç feromon matrisini oluştur."""
        return np.ones((n_cities, n_cities), dtype=float)
    
    @staticmethod
    def _route_length(route: List[int], distance_matrix: np.ndarray) -> float:
        """Rotanın toplam uzunluğunu hesapla."""
        length = 0.0
        for i in range(len(route) - 1):
            length += distance_matrix[route[i], route[i + 1]]
        # Başlangıç noktasına geri dön (kapalı tur)
        length += distance_matrix[route[-1], route[0]]
        return length
    
    def _build_route(self, start_city: int, pheromones: np.ndarray, distance_matrix: np.ndarray) -> List[int]:
        """Bir karınca için rota oluştur."""
        n_cities = distance_matrix.shape[0]
        route = [start_city]
        unvisited = set(range(n_cities))
        unvisited.remove(start_city)
        
        alpha = self.config.alpha
        beta = self.config.beta
        
        while unvisited:
            current = route[-1]
            probs = []
            cities = list(unvisited)
            
            for j in cities:
                tau = pheromones[current, j] ** alpha
                eta = (1.0 / distance_matrix[current, j]) ** beta if distance_matrix[current, j] > 0 else 0.0
                probs.append(tau * eta)
            
            probs = np.array(probs, dtype=float)
            if probs.sum() == 0:
                probs = np.ones_like(probs) / len(probs)
            else:
                probs = probs / probs.sum()
            
            next_city = np.random.choice(cities, p=probs)
            route.append(next_city)
            unvisited.remove(next_city)
        
        return route
    
    def optimize(self, distance_matrix: np.ndarray) -> Tuple[List[int], float, List[float]]:
        """Ana optimizasyon fonksiyonu."""
        n_cities = distance_matrix.shape[0]
        pheromones = self._initialize_pheromones(n_cities)
        
        best_route: Optional[List[int]] = None
        best_distance = float("inf")
        history: List[float] = []
        
        for iteration in range(self.config.n_iterations):
            all_routes: List[List[int]] = []
            all_lengths: List[float] = []
            
            # Her karınca için bir tur oluştur
            for k in range(self.config.n_ants):
                start_city = np.random.randint(0, n_cities)
                route = self._build_route(start_city, pheromones, distance_matrix)
                length = self._route_length(route, distance_matrix)
                all_routes.append(route)
                all_lengths.append(length)
                
                if length < best_distance:
                    best_distance = length
                    best_route = route
            
            # Feromon buharlaşması
            pheromones *= (1.0 - self.config.evaporation_rate)
            
            # Feromon takviyesi
            for route, length in zip(all_routes, all_lengths):
                delta_pheromone = self.config.q / length if length > 0 else 0.0
                for i in range(len(route) - 1):
                    a, b = route[i], route[i + 1]
                    pheromones[a, b] += delta_pheromone
                    pheromones[b, a] += delta_pheromone
                # Başlangıca dönüş kenarı
                a, b = route[-1], route[0]
                pheromones[a, b] += delta_pheromone
                pheromones[b, a] += delta_pheromone
            
            history.append(best_distance)
            
            # İlerleme göster (daha sık)
            if (iteration + 1) % 10 == 0 or (iteration + 1) == 1:
                print(f"İterasyon {iteration + 1}/{self.config.n_iterations}: En iyi mesafe = {best_distance:.2f} km")
        
        assert best_route is not None
        return best_route, best_distance, history

print("✅ AntColonyOptimizer sınıfı tanımlandı!")



## 6. Optimizasyonu Çalıştırma

ACO algoritmasını çalıştırarak en iyi rotayı buluyoruz.


In [ ]:
# ACO parametrelerini ayarla (optimize edilmiş - daha hızlı çalışır)
config = ACOConfig(
    n_ants=10,        # 20'den 10'a düşürüldü (daha hızlı)
    n_iterations=50,  # 100'den 50'ye düşürüldü (daha hızlı, yine de iyi sonuç verir)
    alpha=1.0,
    beta=5.0,
    evaporation_rate=0.5,
    q=100.0,
    random_seed=42
)

print("Algoritma Parametreleri:")
print(f"  - Karınca Sayısı: {config.n_ants}")
print(f"  - İterasyon Sayısı: {config.n_iterations}")
print(f"  - Alpha (Feromon Etkisi): {config.alpha}")
print(f"  - Beta (Mesafe Etkisi): {config.beta}")
print(f"  - Buharlaşma Oranı: {config.evaporation_rate}")
print(f"  - Q (Feromon Sabiti): {config.q}")
print("\nOptimizasyon başlatılıyor...\n")

# Optimizasyonu çalıştır
optimizer = AntColonyOptimizer(config)
best_route, best_distance, history = optimizer.optimize(distance_matrix)

print(f"\n✅ Optimizasyon tamamlandı!")
print(f"📊 En iyi mesafe: {best_distance:.2f} km")
print(f"📈 İyileşme: {((history[0] - history[-1]) / history[0] * 100):.2f}%")



## 7. Sonuçları Görselleştirme

### 7.1. Yakınsama Grafiği

Algoritmanın her iterasyondaki performansını gösterir.



In [ ]:
# Yakınsama grafiği
plt.figure(figsize=(12, 6))
iterations = range(1, len(history) + 1)

plt.plot(iterations, history, linewidth=2, color="#2E86AB", label="En İyi Mesafe")
plt.fill_between(iterations, history, alpha=0.3, color="#2E86AB")

# En iyi değeri işaretle
best_idx = np.argmin(history)
best_value = history[best_idx]
plt.scatter(
    [best_idx + 1],
    [best_value],
    color="#A23B72",
    s=200,
    zorder=5,
    label=f"En İyi: {best_value:.2f} km"
)

plt.xlabel("İterasyon", fontsize=12, fontweight="bold")
plt.ylabel("Mesafe (km)", fontsize=12, fontweight="bold")
plt.title("ACO Algoritması Yakınsama Grafiği", fontsize=14, fontweight="bold", pad=20)
plt.grid(True, alpha=0.3, linestyle="--")
plt.legend(loc="best", fontsize=10)
plt.tight_layout()
plt.show()



### 7.2. Optimize Edilmiş Rota Haritası

En iyi rotayı harita üzerinde görselleştirir.



In [ ]:
# Rota koordinatlarını al
route_coords = []
route_labels = []
for idx in best_route:
    school_name = school_names[idx]
    coords = school_coordinates[school_name]
    route_coords.append([coords["lng"], coords["lat"]])
    route_labels.append(school_name)

# Başlangıç noktasına geri dön (kapalı tur)
route_coords.append(route_coords[0])
route_coords = np.array(route_coords)

# Rota haritasını çiz
fig, ax = plt.subplots(figsize=(14, 10))

# Rota çizgisi
ax.plot(
    route_coords[:, 0],
    route_coords[:, 1],
    "o-",
    linewidth=3,
    markersize=12,
    color="#E63946",
    markerfacecolor="#F1FAEE",
    markeredgewidth=2,
    markeredgecolor="#E63946",
    label="Rota",
    zorder=2,
)

# Okulları numaralandır
for i, (idx, school_name) in enumerate(zip(best_route, route_labels)):
    coords = school_coordinates[school_name]
    ax.annotate(
        f"{i+1}",
        xy=(coords["lng"], coords["lat"]),
        xytext=(5, 5),
        textcoords="offset points",
        fontsize=10,
        fontweight="bold",
        color="white",
        bbox=dict(boxstyle="round,pad=0.3", facecolor="#E63946", alpha=0.8),
        zorder=3,
    )

# Başlangıç noktasını özel işaretle
start_coords = school_coordinates[route_labels[0]]
ax.scatter(
    [start_coords["lng"]],
    [start_coords["lat"]],
    s=300,
    color="#06FF00",
    marker="*",
    edgecolors="black",
    linewidths=2,
    zorder=4,
    label="Başlangıç",
)

ax.set_xlabel("Boylam (Longitude)", fontsize=12, fontweight="bold")
ax.set_ylabel("Enlem (Latitude)", fontsize=12, fontweight="bold")
ax.set_title(
    f"Optimize Edilmiş Rota\nToplam Mesafe: {best_distance:.2f} km",
    fontsize=14,
    fontweight="bold",
    pad=20,
)
ax.grid(True, alpha=0.3, linestyle="--")
ax.legend(loc="best", fontsize=10)
ax.set_aspect("equal", adjustable="box")
plt.tight_layout()
plt.show()



### 7.3. Rota Detayları ve İstatistikler

Optimize edilmiş rotanın detaylı bilgileri.


In [ ]:
print("=" * 60)
print("OPTİMİZE EDİLMİŞ ROTA DETAYLARI")
print("=" * 60)
print(f"\nToplam Mesafe: {best_distance:.2f} km\n")
print("Rota Sırası:")
print("-" * 60)

for i, idx in enumerate(best_route):
    school_name = school_names[idx]
    coords = school_coordinates[school_name]
    print(f"{i+1:2d}. {school_name:40s} ({coords['lat']:.4f}, {coords['lng']:.4f})")

# Başlangıç noktasına geri dön
print(f"{len(best_route)+1:2d}. {school_names[best_route[0]]:40s} (Başlangıç)")
print("=" * 60)

# İstatistikler
print("\n📊 İSTATİSTİKLER")
print("-" * 60)
print(f"Başlangıç Mesafesi: {history[0]:.2f} km")
print(f"En İyi Mesafe:      {history[-1]:.2f} km")
print(f"Ortalama Mesafe:    {np.mean(history):.2f} km")
print(f"İyileşme:           {((history[0] - history[-1]) / history[0] * 100):.2f}%")
print("=" * 60)

